In [60]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
import numpy as np
import random 
from sklearn import preprocessing

In [170]:
def random_sample(data,k):
    return data[data['hash'].isin(random.sample((list(set(data.hash))), k))]
def generate_hash_data(hash_data):
    number_of_records = len(list(hash_data.vmean)) 
    record_number = 0
    new_gen = pd.DataFrame(columns=list(hash_data.columns)[1:])
    for i in range (number_of_records):
        _hash_data = (list(hash_data.hash))[0]
        _trajectory_id = "{}_{}".format(list(hash_data.trajectory_id)[i], record_number)
        if hash_data.x_entry.iloc[i] != hash_data.x_exit.iloc[i] or hash_data.y_entry.iloc[i] != hash_data.y_exit.iloc[i]:
            _time_entry = hash_data.time_entry.iloc[i]
            _time_exit = hash_data.time_exit.iloc[i]
            _v_max = hash_data.vmax.iloc[i]
            _v_min = hash_data.vmin.iloc[i]
            _v_mean = hash_data.vmean.iloc[i] 
            _x_entry = hash_data.x_entry.iloc[i]
            _x_exit = hash_data.x_exit.iloc[i]
            _y_entry = hash_data.y_entry.iloc[i]
            _y_exit = hash_data.y_exit.iloc[i]
            rec = [_hash_data,_trajectory_id,_time_entry,_time_exit,_v_max,_v_min,_v_mean,_x_entry,_y_entry,_x_exit,_y_exit]
            new_record = pd.Series(dict(zip(list(hash_data.columns)[1:], rec)))
            new_gen = new_gen.append(new_record, ignore_index=True)
            record_number += 1
            _trajectory_id = "{}_{}".format(list(hash_data.trajectory_id)[i], record_number)
            if i != number_of_records - 1:
                _time_entry = hash_data.time_exit.iloc[i]
                _time_exit = hash_data.time_entry.iloc[i+1]
                _v_max = _v_min =_v_mean = 0
                _x_entry = hash_data.x_exit.iloc[i]
                _x_exit = hash_data.x_entry.iloc[i+1]
                _y_entry = hash_data.y_exit.iloc[i]
                _y_exit = hash_data.y_entry.iloc[i+1]
                rec = [_hash_data,_trajectory_id,_time_entry,_time_exit,_v_max,_v_min,_v_mean,_x_entry,_y_entry,_x_exit,_y_exit]
                new_record = pd.Series(dict(zip(list(hash_data.columns)[1:], rec)))
                new_gen = new_gen.append(new_record, ignore_index=True)
                record_number += 1
                _trajectory_id = "{}_{}".format(list(hash_data.trajectory_id)[i], record_number)
                
        else:
            if i != number_of_records -1:
                _time_entry = hash_data.time_entry.iloc[i]
                _time_exit = hash_data.time_entry.iloc[i+1]
                _v_max = _v_min =_v_mean = 0
                _x_entry = hash_data.x_entry.iloc[i]
                _x_exit = hash_data.x_entry.iloc[i+1]
                _y_entry = hash_data.y_entry.iloc[i]
                _y_exit = hash_data.y_entry.iloc[i+1]
                rec = [_hash_data,_trajectory_id,_time_entry,_time_exit,_v_max,_v_min,_v_mean,_x_entry,_y_entry,_x_exit,_y_exit]
                new_record = pd.Series(dict(zip(list(hash_data.columns)[1:], rec)))
                new_gen = new_gen.append(new_record, ignore_index=True)
                record_number += 1
    return new_gen

# Generates all data given a subset of the main dataset -> uses generate_hash_data
def generate_data(data):
    progress_tick = 0
    count1 = 0
    id_names = []
    for i in range(len(data)-1):
        if (data['hash'][i] != data['hash'][i+1]):
            id_names.append(data['hash'][i])

#for i in range()dataNew['time_exit']:
    hash_ids = id_names
    generated_data = pd.DataFrame(columns=list(data.columns)[1:])
    for i in hash_ids:
        generated_data = generated_data.append(generate_hash_data(data[data.hash == i])).reset_index().drop('index', axis=1)
        progress_tick += 1
        if progress_tick % 5000 == 0:
            print ('{}% complete'.format(round(progress_tick/ len(hash_ids) * 100 )))
    return generated_data

def prepare_training_data(sample_data):
    hard = sample_data
    hard.time_entry = pd.to_datetime(hard.time_entry)
    hard.time_exit = pd.to_datetime(hard.time_exit)
    t_delta = []
    for i in range(len(hard.time_exit)):
        t_delta.append((hard.time_exit.iloc[i] - hard.time_entry.iloc[i]).total_seconds())
    split = round(len(hard) * .8)
    index = []
    for i in range (len(hard)):
        index.append(i)
    random.shuffle(index)
    train_index = index[:split] 
    test_index = index[split:]
    # vmax , vmin, vmean, x_entery, y_entry, t_del, t_enter, t_exit
    X = np.zeros((len(hard),8))
    for i in range (len(hard)):
        X[i,0] = hard.vmax.iloc[i]
        X[i,1] = hard.vmin.iloc[i]
        X[i,2] = hard.vmean.iloc[i]
        X[i,3] = hard.x_entry.iloc[i]
        X[i,4] = hard.y_entry.iloc[i]
        X[i,5] = t_delta[i]
        X[i,6] = hard.time_entry.iloc[i].hour
        X[i,7] = hard.time_exit.iloc[i].hour
    Y = np.zeros((len(hard),2))
    for i in range(len(hard)):
        Y[i,0] = hard.x_exit.iloc[i]
        Y[i,1] = hard.y_exit.iloc[i]
    X = normalize(X, axis=0, norm='max')    
    X_train = X[train_index, :]
    X_test = X[test_index, :]
    Y_train = Y[train_index, :]
    Y_test = Y[test_index,:]
        
    return (X_train, Y_train, X_test, Y_test)
def Y_to_classification(Y):
    Y_class = np.zeros((len(Y)), dtype=int)
    for i in range (len(Y)):
        temp = 0
        if check_if_in_cbd_p(Y[i][0], Y[i][1]) == 1:
            temp = 1
        Y_class[i] = temp
    return Y_class
def check_stratify(test_points):
    stratify = 0
    for i in test_points:
        if check_if_in_cbd_p(i[0], i[1]) == 0:
            stratify += 1
    return stratify / len(test_points) 
def check_if_in_cbd_x(x):
    return 1 if 3750901.5068 <= x and x <= 3770901.5068 else 0  
def check_if_in_cbd_y(y):
    return 1 if -19268905.6133 <= y and  y <= -19208905.6133 else 0
def check_if_in_cbd_x2(x):
    lab_enc = preprocessing.LabelEncoder()
    num1 =  lab_enc.fit_transform([3750901.5068])
    num2 =  lab_enc.fit_transform([3770901.5068])
    return 1 if num1 <= x and x <= num2 else 0  
def check_if_in_cbd_y2(y):
    lab_enc = preprocessing.LabelEncoder()
    num1 =  lab_enc.fit_transform([-19268905.6133])
    num2 =  lab_enc.fit_transform([-19208905.6133])
    return 1 if num1 <= y and  y <= num2 else 0
def check_if_in_cbd_p2(x,y):
    return 1 if check_if_in_cbd_x2(x) and check_if_in_cbd_y2(y) else 0
def check_if_in_cbd_p(x,y):
    return 1 if check_if_in_cbd_x(x) and check_if_in_cbd_y(y) else 0
def check_accuracy(predicted_x, predicted_y, true_x, true_y):
    acc = 0
    for i in range (len(true_x)):
        if check_if_in_cbd_p(predicted_x[i], predicted_y[i]) == check_if_in_cbd_p(true_x[i], true_y[i]):
            acc += 1
    return (acc / len(true_x))
def check_accuracy2(predicted_x, predicted_y, true_x, true_y):
    acc = 0
    print(true_x)
    lab_enc = preprocessing.LabelEncoder()
    true_x = lab_enc.fit_transform(true_x)
    true_y = lab_enc.fit_transform(true_y)
    print(true_x)
    print(predicted_x)
    for i in range (len(true_x)):
        if check_if_in_cbd_p2(predicted_x[i], predicted_y[i]) == check_if_in_cbd_p2(true_x[i], true_y[i]):
            acc += 1
    return (acc / len(true_x))
def linear_regression(X_train, Y_train, X_test, Y_test):
    model_x = LinearRegression().fit(X_train, Y_train[:,0])
    model_y = LinearRegression().fit(X_train, Y_train[:,1])
    predict_x = model_x.predict(X_test)
    predict_y = model_y.predict(X_test)
    print(predict_x)
    print(check_accuracy(predict_x,predict_y,Y_test[:,0], Y_test[:,1]))
    print(predict_y)
    return (predict_x, predict_y)
def logistic_regression(X_train, Y_train, X_test, Y_test):
    lab_enc = preprocessing.LabelEncoder()
    training_scores_encoded = lab_enc.fit_transform(Y_train)
    logreg1 = LogisticRegression()
    logreg1.fit(X_train,training_scores_encoded)
    predict_x=logreg1.predict(X_test)
    print(check_accuracy2(predict_x,predict_y,Y_test[:,0], Y_test[:,1]))

In [134]:
def prepare_data(sample_data):
    hard = sample_data
    hard.time_entry = pd.to_datetime(hard.time_entry)
    hard.time_exit = pd.to_datetime(hard.time_exit)
    t_delta = []
    for i in range(len(hard.time_exit)):
        t_delta.append((hard.time_exit.iloc[i] - hard.time_entry.iloc[i]).total_seconds())

    # vmax , vmin, vmean, x_entery, y_entry, t_del, t_enter, t_exit
    X = np.zeros((len(hard),8))
    for i in range (len(hard)):
        X[i,0] = hard.vmax.iloc[i]
        X[i,1] = hard.vmin.iloc[i]
        X[i,2] = hard.vmean.iloc[i]
        X[i,3] = hard.x_entry.iloc[i]
        X[i,4] = hard.y_entry.iloc[i]
        X[i,5] = t_delta[i]
        X[i,6] = hard.time_entry.iloc[i].hour
        X[i,7] = hard.time_exit.iloc[i].hour
    Y = np.zeros((len(hard),2))
    for i in range(len(hard)):
        Y[i,0] = hard.x_exit.iloc[i]
        Y[i,1] = hard.y_exit.iloc[i]
    X = normalize(X, axis=0, norm='max')    
        
    return (X,Y)

In [279]:
data_pre = pd.read_csv('data_test.csv').fillna(0)

In [296]:
len(exit2[0])

281131

In [280]:
count = 0
trajectory_id = []
for i in range(len(data_pre)):
    if ((data_pre['x_exit'][i] == 0.0)&(data_pre['y_exit'][i] == 0.0)):
        trajectory_id.append(data_pre['trajectory_id'][i])
        count+=1
        
count
len(trajectory_id)

33515

In [281]:
indexList = []
count = 0
for n in data_pre['time_exit']:
    if ((n[0]=='1')):
        if ((n[1]=='5')):
            indexList.append(count)
        if ((n=='14:00:00')):
            indexList.append(count)
    count+=1


In [282]:
data15pm = data_pre.loc[indexList]

In [284]:
len(data15pm)

33515

In [285]:
data15pm = data15pm.reset_index()

In [294]:
data15pm

,index,Unnamed: 0,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,target,id
0,4,4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:03:32,15:10:32,0.0,0.0,0.0,3.773118e+06,-1.914490e+07,0.0,0.0,0,traj_00032f51796fd5437b238e3a9823d13d_31_5
1,7,7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,15:29:09,15:30:56,0.0,0.0,0.0,3.769978e+06,-1.934136e+07,0.0,0.0,0,traj_000479418b5561ab694a2870cc04fd43_25_10
2,10,10,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,15:26:08,15:26:08,0.0,0.0,0.0,3.757468e+06,-1.923860e+07,0.0,0.0,0,traj_000506a39775e5bca661ac80e3f466eb_29_5
3,13,13,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,15:35:18,15:42:05,0.0,0.0,0.0,3.760505e+06,-1.935500e+07,0.0,0.0,0,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4
4,17,17,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,14:54:07,15:05:14,0.0,0.0,0.0,3.766319e+06,-1.917013e+07,0.0,0.0,0,traj_00063a4f6c12e1e4de7d876580620667_3_4
5,23,23,0006535be25bb52dd06983447880c964_5,traj_0006535be25bb52dd06983447880c964_5_12,15:22:06,15:22:06,0.0,0.0,0.0,3.776037e+06,-1.916993e+07,0.0,0.0,0,traj_0006535be25bb52dd06983447880c964_5_12
6,27,27,0006f84bb33ec929d1cda7686f861d0a_31,traj_0006f84bb33ec929d1cda7686f861d0a_31_3,15:00:12,15:00:12,0.0,0.0,0.0,3.760009e+06,-1.923361e+07,0.0,0.0,0,traj_0006f84bb33ec929d1cda7686f861d0a_31_3
7,31,31,00093ae562586aed0e053b8431e8ace4_23,traj_00093ae562586aed0e053b8431e8ace4_23_10,15:53:09,15:53:09,0.0,0.0,0.0,3.756665e+06,-1.914692e+07,0.0,0.0,0,traj_00093ae562586aed0e053b8431e8ace4_23_10
8,34,34,000c739e444a70e1804d757a0580caaa_31,traj_000c739e444a70e1804d757a0580caaa_31_3,15:41:31,15:41:31,0.0,0.0,0.0,3.768082e+06,-1.922134e+07,0.0,0.0,0,traj_000c739e444a70e1804d757a0580caaa_31_3
9,39,39,000d479078af08618bddc7f09082b8c3_11,traj_000d479078af08618bddc7f09082b8c3_11_6,14:58:06,15:03:40,0.0,0.0,0.0,3.766505e+06,-1.920176e+07,0.0,0.0,0,traj_000d479078af08618bddc7f09082b8c3_11_6


In [293]:
values=[]
for i in range(len(data15pm)):
    value = check_if_in_cbd_p(data15pm['x_exit'][i],data15pm['y_exit'][i])
    print(data15pm['x_exit'][i])
    values.append(value)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


...

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [292]:
values

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [290]:
data15pm['target'] = values
data15pm['id'] = trajectory_id

In [291]:
data15pm

,index,Unnamed: 0,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,target,id
0,4,4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:03:32,15:10:32,0.0,0.0,0.0,3.773118e+06,-1.914490e+07,0.0,0.0,0,traj_00032f51796fd5437b238e3a9823d13d_31_5
1,7,7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,15:29:09,15:30:56,0.0,0.0,0.0,3.769978e+06,-1.934136e+07,0.0,0.0,0,traj_000479418b5561ab694a2870cc04fd43_25_10
2,10,10,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,15:26:08,15:26:08,0.0,0.0,0.0,3.757468e+06,-1.923860e+07,0.0,0.0,0,traj_000506a39775e5bca661ac80e3f466eb_29_5
3,13,13,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,15:35:18,15:42:05,0.0,0.0,0.0,3.760505e+06,-1.935500e+07,0.0,0.0,0,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4
4,17,17,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,14:54:07,15:05:14,0.0,0.0,0.0,3.766319e+06,-1.917013e+07,0.0,0.0,0,traj_00063a4f6c12e1e4de7d876580620667_3_4
5,23,23,0006535be25bb52dd06983447880c964_5,traj_0006535be25bb52dd06983447880c964_5_12,15:22:06,15:22:06,0.0,0.0,0.0,3.776037e+06,-1.916993e+07,0.0,0.0,0,traj_0006535be25bb52dd06983447880c964_5_12
6,27,27,0006f84bb33ec929d1cda7686f861d0a_31,traj_0006f84bb33ec929d1cda7686f861d0a_31_3,15:00:12,15:00:12,0.0,0.0,0.0,3.760009e+06,-1.923361e+07,0.0,0.0,0,traj_0006f84bb33ec929d1cda7686f861d0a_31_3
7,31,31,00093ae562586aed0e053b8431e8ace4_23,traj_00093ae562586aed0e053b8431e8ace4_23_10,15:53:09,15:53:09,0.0,0.0,0.0,3.756665e+06,-1.914692e+07,0.0,0.0,0,traj_00093ae562586aed0e053b8431e8ace4_23_10
8,34,34,000c739e444a70e1804d757a0580caaa_31,traj_000c739e444a70e1804d757a0580caaa_31_3,15:41:31,15:41:31,0.0,0.0,0.0,3.768082e+06,-1.922134e+07,0.0,0.0,0,traj_000c739e444a70e1804d757a0580caaa_31_3
9,39,39,000d479078af08618bddc7f09082b8c3_11,traj_000d479078af08618bddc7f09082b8c3_11_6,14:58:06,15:03:40,0.0,0.0,0.0,3.766505e+06,-1.920176e+07,0.0,0.0,0,traj_000d479078af08618bddc7f09082b8c3_11_6


In [165]:
train_data = pd.read_csv('clean_data_V3.csv').fillna(0)

In [167]:
training_data = prepare_data(train_data)

In [171]:
exit2 = linear_regression(training_data[0], training_data[1], test_data[0], test_data[1])

[3771993.82770132 3771682.0790945  3771760.02349881 ... 3768414.76168701
 3768415.09507393 3768018.56089845]
0.8636934382903344
[-19114823.69694065 -19153834.00223584 -19152467.86777438 ...
 -19153131.97503354 -19153636.87681006 -19157052.80739871]


In [172]:
data_pre['x_exit'] = exit2[0]
data_pre['y_exit'] = exit2[1]

In [132]:
data3 = pd.read_csv('data_test.csv').fillna(0)

In [133]:
csv_new2 = generate_data(data3)
csv_new2.to_csv('clean_test_V3.csv')

15% complete
30% complete
45% complete
60% complete
75% complete
90% complete


In [299]:
data3 = pd.read_csv('clean_test_V3.csv').fillna(0)


In [304]:
test_data = prepare_data(data3)

In [305]:
len(test_data[0])

281131

In [313]:
data3

,Unnamed: 0,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
0,0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0_0,2019-05-08 11:43:17,2019-05-08 11:50:17,0.00,0.00,0.000,3.773413e+06,-1.909828e+07,3.771994e+06,-1.911482e+07
1,1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0_1,2019-05-08 11:50:17,2019-05-08 12:21:37,0.00,0.00,0.000,3.773111e+06,-1.914508e+07,3.771682e+06,-1.915383e+07
2,2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2_2,2019-05-08 12:21:37,2019-05-08 12:34:27,0.00,0.00,0.000,3.773199e+06,-1.914354e+07,3.771760e+06,-1.915247e+07
3,3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3_3,2019-05-08 12:34:27,2019-05-08 13:14:11,0.00,0.00,0.000,3.763760e+06,-1.921342e+07,3.763451e+06,-1.921212e+07
4,4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3_4,2019-05-08 13:14:11,2019-05-08 13:25:33,0.00,0.00,0.000,3.771757e+06,-1.911092e+07,3.770523e+06,-1.912554e+07
5,5,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4_5,2019-05-08 13:25:33,2019-05-08 13:43:13,0.00,0.00,0.000,3.773385e+06,-1.911344e+07,3.771944e+06,-1.912735e+07
6,6,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4_6,2019-05-08 13:43:13,2019-05-08 15:03:32,0.00,0.00,0.000,3.773131e+06,-1.914465e+07,3.771688e+06,-1.915333e+07
7,7,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5_7,2019-05-08 15:03:32,2019-05-08 15:10:32,0.00,0.00,0.000,3.773118e+06,-1.914490e+07,3.771665e+06,-1.915357e+07
8,8,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_0_0,2019-05-08 08:08:23,2019-05-08 08:20:08,17.29,17.29,17.290,3.771380e+06,-1.933274e+07,3.769851e+06,-1.931373e+07
9,9,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_0_1,2019-05-08 08:20:08,2019-05-08 12:08:23,0.00,0.00,0.000,3.769933e+06,-1.934095e+07,3.768776e+06,-1.931685e+07


In [307]:
print(len(training_data[0]))

1060447


In [308]:
exit1 = linear_regression(training_data[0], training_data[1], test_data[0], test_data[1])

[3771993.82770132 3771682.0790945  3771760.02349881 ... 3768414.76168701
 3768415.09507393 3768018.56089845]
0.8636934382903344
[-19114823.69694065 -19153834.00223584 -19152467.86777438 ...
 -19153131.97503354 -19153636.87681006 -19157052.80739871]


In [309]:
data3['x_exit'] = exit1[0]
data3['y_exit'] = exit1[1]

In [330]:
indexList = []
count = 0
for n in data3['time_exit']:
    if ((n.hour==15)):
        indexList.append(count)
    if ((n.hour==14)&(n.minute == 0)&(n.second ==0)):
        indexList.append(count)
    count+=1

In [326]:
len(indexList)

62212

In [332]:
data15pm = data3.loc[indexList]

In [340]:
data15pm = data15pm.reset_index()

In [344]:
data15pm

,index,Unnamed: 0,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
0,6,6,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4_6,2019-05-08 13:43:13,2019-05-08 15:03:32,0.0,0.0,0.0,3.773131e+06,-1.914465e+07,3.771688e+06,-1.915333e+07
1,7,7,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5_7,2019-05-08 15:03:32,2019-05-08 15:10:32,0.0,0.0,0.0,3.773118e+06,-1.914490e+07,3.771665e+06,-1.915357e+07
2,11,11,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_6_3,2019-05-08 12:13:24,2019-05-08 15:29:09,0.0,0.0,0.0,3.769983e+06,-1.934265e+07,3.768791e+06,-1.931819e+07
3,12,12,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10_4,2019-05-08 15:29:09,2019-05-08 15:30:56,0.0,0.0,0.0,3.769978e+06,-1.934136e+07,3.768745e+06,-1.931758e+07
4,15,15,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_2_2,2019-05-08 11:09:02,2019-05-08 15:26:08,0.0,0.0,0.0,3.755349e+06,-1.916135e+07,3.756191e+06,-1.916952e+07
5,16,16,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5_3,2019-05-08 15:26:08,2019-05-08 15:26:08,0.0,0.0,0.0,3.757468e+06,-1.923860e+07,3.757914e+06,-1.923406e+07
6,18,18,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_1_1,2019-05-08 09:37:39,2019-05-08 15:35:18,0.0,0.0,0.0,3.751349e+06,-1.916284e+07,3.752717e+06,-1.917123e+07
7,19,19,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4_2,2019-05-08 15:35:18,2019-05-08 15:42:05,0.0,0.0,0.0,3.760505e+06,-1.935500e+07,3.760463e+06,-1.933041e+07
8,26,26,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4_6,2019-05-08 14:54:07,2019-05-08 15:05:14,0.0,0.0,0.0,3.766319e+06,-1.917013e+07,3.765701e+06,-1.917579e+07
9,31,31,0006535be25bb52dd06983447880c964_5,traj_0006535be25bb52dd06983447880c964_5_10_4,2019-05-08 13:10:01,2019-05-08 15:22:06,0.0,0.0,0.0,3.769801e+06,-1.935089e+07,3.768611e+06,-1.932522e+07


In [345]:
index = []
for i in range(len(data15pm)-1):
    if (data15pm['hash'][i]==data15pm['hash'][i+1]):
        index.append(i)


In [348]:
len(index)
62212-28704

33508

In [210]:

newTraj = []
for i in range(len(data15pmNew)):
    if (data15pmNew['trajectory_id'][i][-2] == '_'):
        newTraj.append(data15pmNew['trajectory_id'][i][:-2])
    else:
        newTraj.append(data15pmNew['trajectory_id'][i][:-3])
#for i in range(len(data15pmNew)):
data15pmNew['trajectory_id'] = newTraj

    

,level_0,index,Unnamed: 0,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
0,1,7,7,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:03:32,15:10:32,0.0,0.0,0.0,3.773118e+06,-1.914490e+07,3.771665e+06,-1.915357e+07
1,3,12,12,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,15:29:09,15:30:56,0.0,0.0,0.0,3.769978e+06,-1.934136e+07,3.768745e+06,-1.931758e+07
2,5,16,16,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,15:26:08,15:26:08,0.0,0.0,0.0,3.757468e+06,-1.923860e+07,3.757914e+06,-1.923406e+07
3,7,19,19,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,15:35:18,15:42:05,0.0,0.0,0.0,3.760505e+06,-1.935500e+07,3.760463e+06,-1.933041e+07
4,10,32,32,0006535be25bb52dd06983447880c964_5,traj_0006535be25bb52dd06983447880c964_5_12,15:22:06,15:22:06,0.0,0.0,0.0,3.776037e+06,-1.916993e+07,3.774189e+06,-1.917396e+07
5,12,37,37,0006f84bb33ec929d1cda7686f861d0a_31,traj_0006f84bb33ec929d1cda7686f861d0a_31_3,15:00:12,15:00:12,0.0,0.0,0.0,3.760009e+06,-1.923361e+07,3.760137e+06,-1.922951e+07
6,14,41,41,00093ae562586aed0e053b8431e8ace4_23,traj_00093ae562586aed0e053b8431e8ace4_23_10,15:53:09,15:53:09,0.0,0.0,0.0,3.756665e+06,-1.914692e+07,3.757295e+06,-1.915790e+07
7,16,45,45,000c739e444a70e1804d757a0580caaa_31,traj_000c739e444a70e1804d757a0580caaa_31_3,15:41:31,15:41:31,0.0,0.0,0.0,3.768082e+06,-1.922134e+07,3.767197e+06,-1.921801e+07
8,19,61,61,000d6fba12a7d06defe90e3c9162c11e_29,traj_000d6fba12a7d06defe90e3c9162c11e_29_11,15:40:02,15:40:02,0.0,0.0,0.0,3.758716e+06,-1.922511e+07,3.759015e+06,-1.922264e+07
9,22,93,93,000feace3a33be9245e2783cdff467a8_15,traj_000feace3a33be9245e2783cdff467a8_15_12,15:21:42,15:38:48,0.0,0.0,0.0,3.772423e+06,-1.916282e+07,3.771045e+06,-1.916854e+07


In [193]:
index = []
for i in range(len(data15pm)-1):
    if (data15pm['hash'][i]==data15pm['hash'][i+1]):
        index.append(i+1)
    

In [196]:
data15pmNew = data15pm.loc[index]

In [ ]:
data15pmNew = data15pmNew.reset_index()

In [211]:
data15pmNew['target'] = 0

In [226]:
values=[]
for i in range(len(data15pmNew)):
    value = check_if_in_cbd_p(data15pmNew['x_exit'][i],data15pmNew['y_exit'][i])
    values.append(value)


In [227]:
data15pmNew['target'] = values

In [232]:
data15pmNew['id'] = data15pmNew['trajectory_id']

In [233]:
finaldata = data15pmNew[['id', 'target']].copy()

In [236]:
finaldata = finaldata.set_index('id')

In [219]:
data15pmNew

,level_0,index,Unnamed: 0,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,target
0,1,7,7,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:03:32,15:10:32,0.0,0.0,0.0,3.773118e+06,-1.914490e+07,3.771665e+06,-1.915357e+07,0
1,3,12,12,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,15:29:09,15:30:56,0.0,0.0,0.0,3.769978e+06,-1.934136e+07,3.768745e+06,-1.931758e+07,0
2,5,16,16,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,15:26:08,15:26:08,0.0,0.0,0.0,3.757468e+06,-1.923860e+07,3.757914e+06,-1.923406e+07,0
3,7,19,19,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,15:35:18,15:42:05,0.0,0.0,0.0,3.760505e+06,-1.935500e+07,3.760463e+06,-1.933041e+07,0
4,10,32,32,0006535be25bb52dd06983447880c964_5,traj_0006535be25bb52dd06983447880c964_5_12,15:22:06,15:22:06,0.0,0.0,0.0,3.776037e+06,-1.916993e+07,3.774189e+06,-1.917396e+07,0
5,12,37,37,0006f84bb33ec929d1cda7686f861d0a_31,traj_0006f84bb33ec929d1cda7686f861d0a_31_3,15:00:12,15:00:12,0.0,0.0,0.0,3.760009e+06,-1.923361e+07,3.760137e+06,-1.922951e+07,0
6,14,41,41,00093ae562586aed0e053b8431e8ace4_23,traj_00093ae562586aed0e053b8431e8ace4_23_10,15:53:09,15:53:09,0.0,0.0,0.0,3.756665e+06,-1.914692e+07,3.757295e+06,-1.915790e+07,0
7,16,45,45,000c739e444a70e1804d757a0580caaa_31,traj_000c739e444a70e1804d757a0580caaa_31_3,15:41:31,15:41:31,0.0,0.0,0.0,3.768082e+06,-1.922134e+07,3.767197e+06,-1.921801e+07,0
8,19,61,61,000d6fba12a7d06defe90e3c9162c11e_29,traj_000d6fba12a7d06defe90e3c9162c11e_29_11,15:40:02,15:40:02,0.0,0.0,0.0,3.758716e+06,-1.922511e+07,3.759015e+06,-1.922264e+07,0
9,22,93,93,000feace3a33be9245e2783cdff467a8_15,traj_000feace3a33be9245e2783cdff467a8_15_12,15:21:42,15:38:48,0.0,0.0,0.0,3.772423e+06,-1.916282e+07,3.771045e+06,-1.916854e+07,0


In [238]:
finaldata.to_csv('ey.csv')

In [218]:
for i in range(len(data15pmNew)):
    if (data15pmNew['target'][i]==1):
        print ("sdafds")

In [242]:
ey = pd.read_csv('ey.csv').fillna(0)

In [245]:
ey = ey.head(9292)

In [246]:
ey.to_csv('ey.csv')

In [249]:
ey.head(9292).to_csv('2.csv')